#Session 4 - RNN are dead, and their renewed relevance
---

### Assignment: 

1) Refer to the file we wrote in the class [Link](https://colab.research.google.com/drive/1-xwX32O0WYOqcCROJnnJiSdzScPCudAM?usp=sharing): 

  > Rewrite this code, but this time remove RNN and add LSTM. 

---

## Solution for Question 1 of Assignment

## Replacing RNN with LSTM
---

**Improvements:**

1) Used Padded sequences for variable length inputs

2) Used Adam optimizer

**Results:**

1) **Test Loss: 0.423**

2) **Test Accuracy: 83.45%**

In [1]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [2]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   1%|          | 606k/84.1M [00:00<00:15, 5.33MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 79.9MB/s]


In [3]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [4]:
print(vars(train_data.examples[0]))

{'text': ['Who', 'would', "n't", 'want', 'to', 'go', 'on', 'road', 'trip', 'with', 'Peter', 'Falk', '?', 'That', 'guy', "'s", 'right', 'eye', 'has', 'more', 'character', 'than', 'most', 'actors', 'today', '.', 'This', 'is', 'the', 'kind', 'of', 'funny', 'and', 'touching', 'movie', 'we', 'are', 'all', 'looking', 'for', 'as', 'a', 'counterbalance', 'to', 'all', 'the', 'bombastic', 'special', 'effects', 'bores', '.', 'Women', 'are', 'going', 'to', 'love', 'it', 'for', 'all', 'the', 'wake', '-', 'up', 'romance', 'advice', 'for', 'men', ',', 'and', 'men', 'will', 'love', 'it', 'for', 'its', 'spot', '-', 'on', 'father', '/', 'son', 'character', 'study', '--', 'one', 'great', 'little', 'scene', 'after', 'another', '.', 'And', 'it', 'has', 'just', 'enough', 'of', 'an', 'edge', 'to', 'be', 'a', 'true', 'indie', 'find', '.', 'Obviously', 'this', 'is', 'a', 'labor', 'of', 'love', 'for', 'Paul', 'Reiser', 'who', 'understands', 'what', 'it', "'s", 'like', 'to', 'be', 'both', 'a', 'father', 'and', '

In [5]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [6]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [7]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [8]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [9]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 202995), (',', 192589), ('.', 166220), ('a', 109823), ('and', 109605), ('of', 100631), ('to', 94086), ('is', 76261), ('in', 61475), ('I', 54572), ('it', 53798), ('that', 49288), ('"', 44123), ("'s", 43366), ('this', 42618), ('-', 36817), ('/><br', 35673), ('was', 35263), ('as', 30534), ('with', 29957)]


In [10]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


In [11]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [12]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True, 
    device = device)

In [13]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=pad_idx)
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]

        #pack sequence
        # lengths need to be on CPU!
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        
        #unpack sequence
        #output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        # print(output[-1,:,:].shape)
        # print(hidden.squeeze(0).shape)
        
        # output, (hidden, cell_state) = self.lstm(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        #assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        y = self.fc(hidden.squeeze(0))
        
        return y

In [14]:

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, PAD_IDX)

In [15]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,867,049 trainable parameters


In [16]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters()) #, lr=1e-3)

In [17]:
criterion = nn.BCEWithLogitsLoss()

In [18]:
model = model.to(device)
criterion = criterion.to(device)

In [19]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [20]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()

        text, text_lengths = batch.text
                
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [24]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            text, text_lengths = batch.text

            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [25]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [26]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 9s
	Train Loss: 0.583 | Train Acc: 69.05%
	 Val. Loss: 0.537 |  Val. Acc: 72.76%
Epoch: 02 | Epoch Time: 0m 8s
	Train Loss: 0.519 | Train Acc: 73.76%
	 Val. Loss: 0.539 |  Val. Acc: 74.14%
Epoch: 03 | Epoch Time: 0m 9s
	Train Loss: 0.405 | Train Acc: 82.12%
	 Val. Loss: 0.468 |  Val. Acc: 79.13%
Epoch: 04 | Epoch Time: 0m 9s
	Train Loss: 0.347 | Train Acc: 85.20%
	 Val. Loss: 0.469 |  Val. Acc: 77.53%
Epoch: 05 | Epoch Time: 0m 9s
	Train Loss: 0.293 | Train Acc: 88.29%
	 Val. Loss: 0.439 |  Val. Acc: 80.56%
Epoch: 06 | Epoch Time: 0m 9s
	Train Loss: 0.264 | Train Acc: 89.78%
	 Val. Loss: 0.423 |  Val. Acc: 83.43%
Epoch: 07 | Epoch Time: 0m 9s
	Train Loss: 0.223 | Train Acc: 91.42%
	 Val. Loss: 0.474 |  Val. Acc: 81.06%
Epoch: 08 | Epoch Time: 0m 9s
	Train Loss: 0.168 | Train Acc: 93.73%
	 Val. Loss: 0.424 |  Val. Acc: 83.81%
Epoch: 09 | Epoch Time: 0m 9s
	Train Loss: 0.114 | Train Acc: 96.09%
	 Val. Loss: 0.479 |  Val. Acc: 84.10%
Epoch: 10 | Epoch Time: 0m 9

In [28]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.423 | Test Acc: 83.45%
